In [1]:
import os
import pandas as pd
import json
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from dotenv import load_dotenv
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

load_dotenv()

True

In [2]:
def load_env():
    db_user = os.getenv('DB_USER')
    db_pass = quote_plus(os.getenv('DB_PASS'))
    db_host = os.getenv('DB_HOST')
    db_port = os.getenv('DB_PORT')
    db_name = os.getenv('DB_NAME')

    db_uri = f'postgresql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}'
    engine = create_engine(db_uri)

    return engine

In [3]:
def train_data():
    engine = load_env()
    query = """
    SELECT
        id,
        question_id,
        major_id,
        weight
    FROM gema_question_major_weights
    """
    qmw_df = pd.read_sql(query, engine)

    df = qmw_df.pivot(
        index='major_id',
        columns='question_id',
        values='weight'
    ).reset_index()

    numerical_columns = df.select_dtypes(include=['number']).columns
    scaler = StandardScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

    X = df.drop(columns=['major_id']).values
    y = df['major_id'].values

    k = df.shape[0] // 3
    kmeans = KMeans(n_clusters=k, random_state=42)
    clusters = kmeans.fit_predict(X)

    df['cluster'] = clusters
    df['major_id'] = y

    return df, X, scaler, kmeans

In [4]:
def recommend_majors(answer: list[int]):
    engine = load_env()
    df, X, scaler, kmeans = train_data()
    if len(answer) != X.shape[1]:
        raise ValueError(f"Panjang jawaban siswa ({len(answer)}) tidak sesuai jumlah soal ({X.shape[1]})")
    
    answer_scaled = scaler.transform([answer])
    predicted_cluster = kmeans.predict(answer_scaled)[0]

    cluster_df = df[df['cluster'] == predicted_cluster]
    query = """
    SELECT
        id as major_id,
        name
    FROM gema_majors
    """
    major_df = pd.read_sql(query, engine)
    merged_df = cluster_df.merge(
        major_df,
        left_on='major_id',
        right_on='major_id',
        how='left'
    )
    top_majors = merged_df[['major_id', 'name']].head(3)

    return json.loads(top_majors.to_json(orient='records'))

In [5]:
data = [
    {
        "question_id": "PS001",
        "score": 3
    },
    {
        "question_id": "PS002",
        "score": 4
    },
    {
        "question_id": "PS003",
        "score": 3
    },
    {
        "question_id": "PS004",
        "score": 1
    },
    {
        "question_id": "PS005",
        "score": 4
    },
    {
        "question_id": "PS006",
        "score": 2
    },
    {
        "question_id": "PS007",
        "score": 5
    },
    {
        "question_id": "PS008",
        "score": 1
    },
    {
        "question_id": "PS009",
        "score": 2
    },
    {
        "question_id": "PS010",
        "score": 4
    },
    {
        "question_id": "PS011",
        "score": 5
    },
    {
        "question_id": "PS012",
        "score": 2
    },
    {
        "question_id": "PS013",
        "score": 1
    },
    {
        "question_id": "PS014",
        "score": 1
    },
    {
        "question_id": "PS015",
        "score": 2
    },
    {
        "question_id": "PS016",
        "score": 4
    },
    {
        "question_id": "PS017",
        "score": 3
    },
    {
        "question_id": "PS018",
        "score": 1
    },
    {
        "question_id": "PS019",
        "score": 1
    },
    {
        "question_id": "PS020",
        "score": 1
    },
    {
        "question_id": "PS021",
        "score": 2
    },
    {
        "question_id": "PS022",
        "score": 5
    },
    {
        "question_id": "PS023",
        "score": 1
    },
    {
        "question_id": "PS024",
        "score": 4
    },
    {
        "question_id": "PS025",
        "score": 5
    },
    {
        "question_id": "PS026",
        "score": 1
    },
    {
        "question_id": "PS027",
        "score": 5
    },
    {
        "question_id": "PS028",
        "score": 1
    },
    {
        "question_id": "PS029",
        "score": 5
    },
    {
        "question_id": "PS030",
        "score": 3
    },
    {
        "question_id": "PS031",
        "score": 1
    },
    {
        "question_id": "PS032",
        "score": 2
    },
    {
        "question_id": "PS033",
        "score": 4
    },
    {
        "question_id": "PS034",
        "score": 1
    },
    {
        "question_id": "PS035",
        "score": 4
    },
    {
        "question_id": "PS036",
        "score": 2
    },
    {
        "question_id": "PS037",
        "score": 5
    },
    {
        "question_id": "PS038",
        "score": 3
    },
    {
        "question_id": "PS039",
        "score": 1
    },
    {
        "question_id": "PS040",
        "score": 4
    },
    {
        "question_id": "PS041",
        "score": 4
    },
    {
        "question_id": "PS042",
        "score": 1
    },
    {
        "question_id": "PS043",
        "score": 3
    },
    {
        "question_id": "PS044",
        "score": 5
    },
    {
        "question_id": "PS045",
        "score": 2
    },
    {
        "question_id": "PS046",
        "score": 2
    },
    {
        "question_id": "PS047",
        "score": 2
    },
    {
        "question_id": "PS048",
        "score": 3
    },
    {
        "question_id": "PS049",
        "score": 1
    },
    {
        "question_id": "AI001",
        "score": 4
    },
    {
        "question_id": "AI002",
        "score": 1
    },
    {
        "question_id": "AI003",
        "score": 2
    },
    {
        "question_id": "AI004",
        "score": 5
    },
    {
        "question_id": "AI005",
        "score": 2
    },
    {
        "question_id": "AI006",
        "score": 1
    },
    {
        "question_id": "AI007",
        "score": 2
    },
    {
        "question_id": "AI008",
        "score": 3
    },
    {
        "question_id": "AI009",
        "score": 5
    },
    {
        "question_id": "AI010",
        "score": 3
    },
    {
        "question_id": "AI011",
        "score": 3
    },
    {
        "question_id": "AI012",
        "score": 1
    },
    {
        "question_id": "AI013",
        "score": 5
    },
    {
        "question_id": "AI014",
        "score": 3
    },
    {
        "question_id": "AI015",
        "score": 2
    },
    {
        "question_id": "AI016",
        "score": 3
    },
    {
        "question_id": "AI017",
        "score": 4
    },
    {
        "question_id": "AI018",
        "score": 3
    },
    {
        "question_id": "AI019",
        "score": 3
    },
    {
        "question_id": "AI020",
        "score": 3
    },
    {
        "question_id": "AI021",
        "score": 3
    },
    {
        "question_id": "AI022",
        "score": 1
    },
    {
        "question_id": "AI023",
        "score": 3
    },
    {
        "question_id": "AI024",
        "score": 3
    },
    {
        "question_id": "AI025",
        "score": 2
    },
    {
        "question_id": "AI026",
        "score": 1
    },
    {
        "question_id": "AI027",
        "score": 2
    },
    {
        "question_id": "AI028",
        "score": 3
    },
    {
        "question_id": "AI029",
        "score": 5
    },
    {
        "question_id": "AI030",
        "score": 4
    },
    {
        "question_id": "AI031",
        "score": 5
    },
    {
        "question_id": "AI032",
        "score": 2
    },
    {
        "question_id": "AI033",
        "score": 3
    },
    {
        "question_id": "AI034",
        "score": 1
    },
    {
        "question_id": "AI035",
        "score": 3
    },
    {
        "question_id": "AI036",
        "score": 2
    },
    {
        "question_id": "AI037",
        "score": 2
    },
    {
        "question_id": "AI038",
        "score": 1
    },
    {
        "question_id": "AI039",
        "score": 4
    },
    {
        "question_id": "AI040",
        "score": 2
    },
    {
        "question_id": "AI041",
        "score": 2
    },
    {
        "question_id": "AI042",
        "score": 5
    },
    {
        "question_id": "AI043",
        "score": 3
    },
    {
        "question_id": "AI044",
        "score": 1
    },
    {
        "question_id": "AI045",
        "score": 3
    },
    {
        "question_id": "AI046",
        "score": 4
    },
    {
        "question_id": "AI047",
        "score": 5
    },
    {
        "question_id": "AI048",
        "score": 2
    },
    {
        "question_id": "AI049",
        "score": 3
    }
]

answer = [answer['score'] for answer in data]
majors = recommend_majors(answer)
print("Recommended Majors:", majors)

c:\Users\willy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Recommended Majors: [{'major_id': 'BK008', 'name': 'BAHASA KOREA'}, {'major_id': 'MA043', 'name': 'MANAJEMEN'}, {'major_id': 'PA049', 'name': 'PARIWISATA'}]
